1. (2x) Gaussian process regression for electronic band gap, using principal component analysis to select features.


(a) Read in the JSON file with the cleaned electronic band gap data for rocksalt structure materials, and the JSON file with the elemental properties. Convert both to dictionary-like objects.

In [1]:
import json
from google.colab import drive
drive.mount('/content/drive')

f = open("/content/drive/MyDrive/Colab Notebooks/rocksalt_Egap_insulators_AFLOW.json", "r+")
zincblende_EgapClean = json.load(f)
f.close()
print(zincblende_EgapClean)  #  already dictionary-like object

f2 = open("/content/drive/MyDrive/Colab Notebooks/Chemical_element_data.json", "r+")
Chemical_element_data = json.load(f2)
f2.close()
print(Chemical_element_data)  #  already dictionary-like object

drive.flush_and_unmount()

Mounted at /content/drive
[{'compound': 'Ag1Br1', 'auid': 'aflow:1ffe490975e7aeeb', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Ag1Br1_ICSD_52246', 'spacegroup_relax': 225, 'Pearson_symbol_relax': 'cF8', 'species': ['Ag', 'Br'], 'Egap': 1.5727, 'Egap_type': 'insulator-indirect', 'aflow_prototype_label_relax': 'AB_cF8_225_a_b'}, {'compound': 'Ag1Cl1', 'auid': 'aflow:1b9d91d05f2e4c8c', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Ag1Cl1_ICSD_64734', 'spacegroup_relax': 225, 'Pearson_symbol_relax': 'cF8', 'species': ['Ag', 'Cl'], 'Egap': 1.9714, 'Egap_type': 'insulator-indirect', 'aflow_prototype_label_relax': 'AB_cF8_225_a_b'}, {'compound': 'Ag1F1', 'auid': 'aflow:bcee7bb7be81cfb8', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Ag1F1_ICSD_18008', 'spacegroup_relax': 225, 'Pearson_symbol_relax': 'cF8', 'species': ['Ag', 'F'], 'Egap': 1.0812, 'Egap_type': 'insulator-indirect', 'aflow_prototype_label_relax': 'AB_cF8_225_a_b'}, {'compound': 'Ag1I1', 'auid': 'aflow:382493c94d10

(b) Generate the feature vectors based on the means and differences of the electronegativities, ionization energies, valences and atomic masses (8 features, see example in Lecture 17), and the list with the values of the band gap. Generate the training and test sets.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

x_list = []
y_list = []

for datum in zincblende_EgapClean:
    species1 = datum["species"][0]
    species2 = datum["species"][1]
    en_mean = abs(Chemical_element_data[species1]["electronegativity"] + Chemical_element_data[species2]["electronegativity"]) / 2.0
    en_diff = abs(Chemical_element_data[species1]["electronegativity"] - Chemical_element_data[species2]["electronegativity"])
    ie_mean = abs(Chemical_element_data[species1]["first_ionization_energy"] + Chemical_element_data[species2]["first_ionization_energy"]) / 2.0
    ie_diff = abs(Chemical_element_data[species1]["first_ionization_energy"] - Chemical_element_data[species2]["first_ionization_energy"])
    val_mean = abs(Chemical_element_data[species1]["valence"] + Chemical_element_data[species2]["valence"]) / 2.0
    val_diff = abs(Chemical_element_data[species1]["valence"] - Chemical_element_data[species2]["valence"])
    am_mean = abs(Chemical_element_data[species1]["atomic_mass"] + Chemical_element_data[species2]["atomic_mass"]) / 2.0
    am_diff = abs(Chemical_element_data[species1]["atomic_mass"] - Chemical_element_data[species2]["atomic_mass"])
    x_list.append([en_mean, en_diff, ie_mean, ie_diff, val_mean, val_diff, am_mean, am_diff])
    y_list.append(float(datum["Egap"]))

print(x_list)
print(y_list)

x = np.array(x_list)
y = np.array(y_list)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

[[2.3499999999999996, 0.8999999999999999, 936.0, 408.0, 4.0, 6.0, 93.898, 27.964], [2.45, 1.1, 996.0, 528.0, 4.0, 6.0, 71.6685, 72.423], [2.95, 2.1, 1206.0, 948.0, 4.0, 6.0, 63.44, 88.88], [2.2, 0.6000000000000001, 871.0, 278.0, 4.0, 6.0, 117.4, 19.040000000000006], [2.25, 1.5, 988.5, 823.0, 4.0, 2.0, 20.488999999999997, 12.962], [2.2, 2.6, 906.0, 808.0, 4.0, 4.0, 76.68, 121.36000000000001], [1.7, 1.6, 751.0, 498.0, 4.0, 4.0, 84.71300000000001, 105.29400000000001], [1.65, 1.5, 271.5, 461.0, 4.0, 4.0, 108.16, 58.40000000000002], [1.5, 1.2000000000000002, 686.0, 368.0, 4.0, 4.0, 132.485, 9.750000000000014], [2.5, 2.0, 1105.0, 410.0, 4.0, 4.0, 12.506499999999999, 6.987], [2.0, 1.0, 950.0, 100.0, 4.0, 4.0, 20.5395, 23.053000000000004], [1.7999999999999998, 1.9999999999999998, 779.0, 722.0, 4.0, 6.0, 59.506, 40.82], [1.7999999999999998, 1.9999999999999998, 771.0, 738.0, 4.0, 6.0, 82.69800000000001, 5.564000000000007], [2.25, 2.5, 950.0, 720.0, 4.0, 4.0, 28.04, 24.08], [1.75, 1.5, 795.0, 410

(c) Use principal component analysis to find the principal components that explain the largest and second largest variation in the sample (first two principal components).

In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

#  Rescale data
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

#  Fit PCA
pca = PCA(n_components = 2)
pca.fit(x_scaled)

#  Transform data onto first two principal components
x_pca = pca.transform(x_scaled)

print("First principal component: ", pca.components_[0])
print("Second principal component: ", pca.components_[1])

First principal component:  [ 0.39535065  0.53350955  0.43414156  0.43552639 -0.01454952  0.26369702
 -0.33156464 -0.03493381]
Second principal component:  [-0.43119683  0.02585379 -0.01563038  0.19620415 -0.61804665 -0.34005602
 -0.4498253  -0.27330924]


(d) Using the principal components from part (c) as the features, split the sample set into training and test sets. Fit the data using the Gaussian Process Regressor from sci-kit learn with the Matern kernel, using GridSearchCV and Pipeline to optimize the Matern kernel parameter using a grid of nu values of [0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0]. What is the accuracy score for the optimized classifier for the test set?

In [4]:
from google.colab import output
output.no_vertical_scroll()

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.2, random_state = 42)

pipe = Pipeline([("scaler", StandardScaler()), ("gpr", GaussianProcessRegressor(kernel = Matern()))])

param_grid = {"gpr__kernel__nu": [0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0]}

grid_search = GridSearchCV(pipe, param_grid = param_grid, cv = 5, return_train_score = True)
grid_search.fit(x_train, y_train)

print("Best accuracy on training set: ", grid_search.best_score_)
print("Accuracy on test set: ", grid_search.score(x_test, y_test))
print("Best estimator: ", grid_search.best_estimator_)

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The opti

Best accuracy on training set:  -0.2467451285208207
Accuracy on test set:  0.8064160882400848
Best estimator:  Pipeline(steps=[('scaler', StandardScaler()),
                ('gpr',
                 GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=0.5)))])


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
